# AQI Analysis by Province (4-hour intervals)

This notebook loads AQI data from the SQLite database and provides:
- Province selector in a left sidebar
- 4-hour aggregated AQI time-series
- Per-province summaries and plots


In [12]:
import os
import sqlite3
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from IPython.display import display, HTML

DB_PATH = 'data/processed/aqi_history.db'

if not os.path.exists(DB_PATH):
    display(HTML(f"<div style='padding:8px;border:1px solid #f0ad4e;background:#fcf8e3'>"
                 f"<b>Chưa có cơ sở dữ liệu:</b> {DB_PATH}.<br/>"
                 f"Hãy chạy <code>py backend/data_processing.py</code> hoặc <code>run.py</code> để tạo dữ liệu trước rồi chạy lại notebook.</div>"))
    raw_df = pd.DataFrame(columns=['Province','AQI','Date'])
else:
    conn = sqlite3.connect(DB_PATH)
    raw_df = pd.read_sql('SELECT * FROM daily_aqi', conn, parse_dates=['Date'])
    conn.close()

    # Normalize columns
    expected_cols = {'Province', 'AQI', 'Date'}
    missing = expected_cols - set(raw_df.columns)
    if missing:
        raise ValueError(f'Missing required columns: {missing}. Found: {list(raw_df.columns)}')

    raw_df['Province'] = raw_df['Province'].astype(str)
    raw_df['AQI'] = pd.to_numeric(raw_df['AQI'], errors='coerce')
    raw_df = raw_df.dropna(subset=['Province', 'Date'])
    raw_df = raw_df.sort_values('Date').reset_index(drop=True)

raw_df.head()


,Province,AQI,Date


In [13]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML

# Ensure raw_df exists even if previous cell wasn't run
try:
    raw_df
except NameError:
    import os, sqlite3, pandas as pd
    DB_PATH = 'data/processed/aqi_history.db'
    assert os.path.exists(DB_PATH), f'Database not found at {DB_PATH}'
    conn = sqlite3.connect(DB_PATH)
    raw_df = pd.read_sql('SELECT * FROM daily_aqi', conn, parse_dates=['Date'])
    conn.close()
    raw_df['Province'] = raw_df['Province'].astype(str)
    raw_df['AQI'] = pd.to_numeric(raw_df['AQI'], errors='coerce')
    raw_df = raw_df.dropna(subset=['Province', 'Date']).sort_values('Date').reset_index(drop=True)

# Build 4-hour bins
raw_df['Date'] = pd.to_datetime(raw_df['Date'])
raw_df['Date_4h'] = raw_df['Date'].dt.floor('4h')

# Province list
provinces = sorted(raw_df['Province'].dropna().unique().tolist())

# AQI label helper
AQI_LABELS = [
    (0, 50, 'Tốt', '#00e400'),
    (51, 100, 'Trung bình', '#ffff00'),
    (101, 150, 'Kém', '#ff7e00'),
    (151, 200, 'Xấu', '#ff0000'),
    (201, 300, 'Rất xấu', '#8f3f97'),
    (301, np.inf, 'Nguy hại', '#7e0023'),
]

def label_aqi(aqi: float):
    if pd.isna(aqi):
        return ('N/A', '#9e9e9e')
    for lo, hi, name, color in AQI_LABELS:
        if lo <= aqi <= hi:
            return (name, color)
    return ('N/A', '#9e9e9e')

# Sidebar: list provinces with current AQI label (latest)
latest = (raw_df
          .sort_values('Date')
          .groupby('Province', as_index=False)
          .tail(1)
          .set_index('Province'))

def make_sidebar():
    items = []
    for province in provinces:
        aqi_val = latest.loc[province, 'AQI'] if province in latest.index else np.nan
        name, color = label_aqi(aqi_val)
        label_html = f"<b>{province}</b><br/>AQI: {'' if pd.isna(aqi_val) else int(aqi_val)} — <span style='color:{color}'>{name}</span>"
        items.append(widgets.HTML(value=label_html))
    sidebar = widgets.VBox(items, layout=widgets.Layout(width='280px', overflow='auto', border='1px solid #ddd', padding='8px', height='70vh'))
    return sidebar

province_dropdown = widgets.Dropdown(options=provinces, description='Tỉnh', layout=widgets.Layout(width='260px'))

sidebar_header = widgets.HTML(value='<h4>Danh sách tỉnh & nhãn AQI</h4>')
sidebar = widgets.VBox([sidebar_header, province_dropdown, make_sidebar()], layout=widgets.Layout(width='300px'))

content_out = widgets.Output()

app = widgets.HBox([sidebar, content_out], layout=widgets.Layout(align_items='flex-start'))

display(app)


In [14]:
def aggregate_4h(df: pd.DataFrame, province: str) -> pd.DataFrame:
    sub = df[df['Province'] == province].copy()
    if sub.empty:
        return sub
    sub = sub.set_index('Date_4h').sort_index()
    # For each 4-hour bin, use mean AQI; you can change to median or last
    agg = (sub
           .groupby('Date_4h')
           .agg({'AQI': 'mean'})
           .reset_index())
    return agg


def render_province_view(province: str):
    with content_out:
        content_out.clear_output(wait=True)
        agg = aggregate_4h(raw_df, province)
        if agg.empty:
            display(HTML(f'<h3>{province}</h3><p>Không có dữ liệu.</p>'))
            return

        # Latest stats
        latest_row = agg.sort_values('Date_4h').tail(1).iloc[0]
        aqi_val = latest_row['AQI']
        label, color = label_aqi(aqi_val)

        display(HTML(f"""
        <h3>{province}</h3>
        <p><b>AQI gần nhất (4h):</b> <span style='color:{color}'>{aqi_val:.0f} — {label}</span></p>
        """))

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=agg['Date_4h'], y=agg['AQI'], mode='lines+markers', name='AQI'))
        fig.update_layout(
            title=f'AQI 4 giờ - {province}',
            xaxis_title='Thời gian',
            yaxis_title='AQI',
            template='plotly_white',
            height=420
        )
        fig.add_hrect(y0=0, y1=50, fillcolor='#00e400', opacity=0.05, line_width=0)
        fig.add_hrect(y0=51, y1=100, fillcolor='#ffff00', opacity=0.05, line_width=0)
        fig.add_hrect(y0=101, y1=150, fillcolor='#ff7e00', opacity=0.05, line_width=0)
        fig.add_hrect(y0=151, y1=200, fillcolor='#ff0000', opacity=0.05, line_width=0)
        fig.add_hrect(y0=201, y1=300, fillcolor='#8f3f97', opacity=0.05, line_width=0)
        fig.add_hrect(y0=301, y1=max(350, (agg['AQI'].max() or 350)), fillcolor='#7e0023', opacity=0.05, line_width=0)
        fig.show()

# Render once, even if dropdown cell hasn't run yet
try:
    default_province = province_dropdown.value
except NameError:
    default_province = provinces[0] if len(provinces) else None

if default_province is not None:
    render_province_view(default_province)


def on_change(change):
    if change['name'] == 'value' and change['type'] == 'change':
        render_province_view(change['new'])

# Attach observer only if widget exists
try:
    province_dropdown.observe(on_change)
except NameError:
    pass

